In [46]:
# imports
import os
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import numpy as np
import pandas as pd
import nbconvert
import nbformat
import zipfile
import csv
import time

In [77]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])

# global vars
INDEX="notebookindex"
TYPE= "notebook"
COLUMNS = ['nb_id', 'html_url', 'name', 'language', 'markdown', 'comments', 'code']

In [ ]:
# Initialise the elastic search index
init_md_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "text_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'markdown': {'type': 'text', 
                             "analyzer":"text_analyzer"
                            },
        }
    }
}
es.indices.create(index = 'markdown', body = init_index)

In [ ]:
# Initialise the elastic search index
init_md_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "text_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'comments': {'type': 'text', 
                             "analyzer":"text_analyzer"
                            },
        }
    }
}
es.indices.create(index = 'markdown', body = init_index)

In [98]:
# Initialise the elastic search index
init_info_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'html_url': {'type': 'keyword'},
                'name': {'type': 'keyword'},
                'language': {'type': 'keyword'},
        }
    }
}
es.indices.create(index = 'info', body = init_info_index)

{'acknowledged': True, 'index': 'info', 'shards_acknowledged': True}

In [23]:
# tes_df = pd.read_csv('df_bb2733859v_2_1_old.csv')
tes_df = pd.read_pickle('df_bb2733859v_2_1.pkl')
hes_df = tes_df.head()

In [100]:
es_df = tes_df

for i, row in es_df.iterrows():
    t = {
        "html_url": row['html_url'],
        'name': row['name'],
        'language': row['language']
        }
    res = es.index(index="info",id=row['nb_id'], body=t)
    if res['result'] != 'created':
        print(res)

{'_index': 'info', '_type': '_doc', '_id': '120951', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
{'_index': 'info', '_type': '_doc', '_id': '23352', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}


In [19]:
tes_df.head()

,nb_id,html_url,name,language,markdown,comments,code
0,120951,https://github.com/tuckercs/DSI-Projects/blob/...,Fizz Buzz.ipynb,python,[],[],"[import pandas as pd, def fizzbuzz(n):\n if..."
1,23352,https://github.com/Isabella-Dineo/beam_py/blob...,patchbeam_main.ipynb,python,[],"[def correct(x):\n """"""Function to correct f...",[import numpy as np\nimport scipy as sp\nfrom ...
2,73638,https://github.com/wanglongjuan/15-ODE-homewor...,2015750221张杰-checkpoint.ipynb,,"[数学与应用数学班, 张杰 2015750221, **P88——3**, **求初值问题$...",[],[]
3,179265,https://github.com/tyui592/nn_study/blob/f3c37...,hand_write_function_learning.ipynb,python,"[# Artificial Neural Network, ## Object : hand...",[# read hand write function image\nimg = color...,"[from skimage import io, color\nimport matplot..."
4,54493,https://github.com/suhagba/Naive_Bayes/blob/e6...,Tutorial5.ipynb,python,[**Name:** Byaravalli Arun Suhag\n\n**EID:** 5...,[%matplotlib inline\nimport IPython.core.displ...,[%matplotlib inline\nimport IPython.core.displ...


In [97]:
es.search(body={"query": {"match_all": {}}}, index = INDEX)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [],
  'max_score': None,
  'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 0}

In [92]:
es.ping()

True

In [101]:
t = pd.read_pickle('df_bb2733859v_2_1.pkl')
t.head()

,nb_id,html_url,name,language,markdown,comments,code
0,120951,https://github.com/tuckercs/DSI-Projects/blob/...,Fizz Buzz.ipynb,python,[],[],"[import pandas as pd, def fizzbuzz(n):\n if..."
1,23352,https://github.com/Isabella-Dineo/beam_py/blob...,patchbeam_main.ipynb,python,[],"[def correct(x):\n """"""Function to correct f...",[import numpy as np\nimport scipy as sp\nfrom ...
2,73638,https://github.com/wanglongjuan/15-ODE-homewor...,2015750221张杰-checkpoint.ipynb,,"[数学与应用数学班, 张杰 2015750221, **P88——3**, **求初值问题$...",[],[]
3,179265,https://github.com/tyui592/nn_study/blob/f3c37...,hand_write_function_learning.ipynb,python,"[# Artificial Neural Network, ## Object : hand...",[# read hand write function image\nimg = color...,"[from skimage import io, color\nimport matplot..."
4,54493,https://github.com/suhagba/Naive_Bayes/blob/e6...,Tutorial5.ipynb,python,[**Name:** Byaravalli Arun Suhag\n\n**EID:** 5...,[%matplotlib inline\nimport IPython.core.displ...,[%matplotlib inline\nimport IPython.core.displ...
